# 2. Validação dos Dados Gerados

Este notebook executa verificações de consistência na planilha gerada pelo processo anterior.

**Objetivo:** Garantir que não existam erros como nomes não encontrados, mensalidades nulas indevidas ou duplicatas.

**Pré-requisito:** Ter executado com sucesso o notebook `1_Gerar_Nova_Planilha.ipynb`.

### 2.1 Carregamento dos Dados para Validação
Carrega o arquivo gerado (`dados_atualizados.xlsx`) e a planilha original de mensalidades para comparação.

In [ ]:
import pandas as pd

In [ ]:
caminho_dados = '01_Processamento/dados_atualizados.xlsx' # Caminho para a planilha de dados ATUALIZADOS
caminho_mensalidades = '' # Caminho para a planilha de mensalidades

dados = pd.read_excel(caminho_dados, index_col=0)
mensalidades = pd.read_excel(caminho_mensalidades, index_col=0)
mensalidades['Nome'] = mensalidades['Nome'].astype(str).str.strip()

### 2.2 Verificação de Mensalidades Nulas
Verifica se existem pessoas com nome preenchido mas sem valor de mensalidade.

In [ ]:
dados_filtered = dados.dropna(subset=['NOME'])

null_mensalidades_count = dados_filtered['MENSALIDADE'].isnull().sum()

print(f"O número de valores nulos na coluna MENSALIDADE (onde NOME não é NaN) é: {null_mensalidades_count}")

### 2.3 Verificação de Nomes Não Encontrados (Mensalidades -> Dados)
Verifica se há nomes na planilha de mensalidades que não foram encontrados na nossa planilha de dados. Isso pode indicar novos beneficiários ou erros de digitação.

In [ ]:
nomes_mensalidades = (
    mensalidades['Nome']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)
nomes_dados = (
    dados['NOME']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

nomes_mensalidades = nomes_mensalidades[nomes_mensalidades.notna() & (nomes_mensalidades != '')]
nomes_dados = nomes_dados[nomes_dados.notna() & (nomes_dados != '')]

set_dados = set(nomes_dados.tolist())
nomes_nao_encontrados = [nome for nome in nomes_mensalidades.tolist() if nome not in set_dados]

if nomes_nao_encontrados:
    print("Names in 'mensalidades' not found in 'dados':")
    for nome in nomes_nao_encontrados:
        print(nome)
else:
    print("All names in 'mensalidades' are present in 'dados'.")

In [ ]:
mensalidades_aux = mensalidades.reset_index().copy()

mensalidades_aux = mensalidades_aux.rename(columns={'Código': 'Codigo'})

mensalidades_aux['Nome_norm'] = (
    mensalidades_aux['Nome']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)
dados_nomes_norm = (
    dados['NOME']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

mensalidades_aux = mensalidades_aux[mensalidades_aux['Nome_norm'].notna() & (mensalidades_aux['Nome_norm'] != '')]
dados_nomes_norm = dados_nomes_norm[dados_nomes_norm.notna() & (dados_nomes_norm != '')]

set_dados = set(dados_nomes_norm.tolist())

novos = mensalidades_aux[~mensalidades_aux['Nome_norm'].isin(set_dados)].copy()

novos['Codigo'] = novos['Codigo'].astype('string').str.replace(r'\.0$', '', regex=True).str.strip()
novos['Cod da Familia'] = novos['Codigo'].str.replace(r'\D', '', regex=True).str.slice(-6, -3)

def normaliza_familia(codigo):
    if codigo is None or pd.isna(codigo):
        return pd.NA

    if isinstance(codigo, (int, float)):
        try:
            return f"{int(codigo):03d}"
        except (ValueError, TypeError):
            return pd.NA
    texto = str(codigo).strip()
    if texto == '':
        return pd.NA

    if '.' in texto:
        texto = texto.split('.')[0]

    digits = ''.join(ch for ch in texto if ch.isdigit())
    if digits == '':
        return pd.NA

    if len(digits) <= 3:
        return digits.zfill(3)
    return digits[-3:]

if 'CÓD DA FAMILIA' in dados.columns:
    familias_base = dados['CÓD DA FAMILIA']
else:
    dados_reset = dados.reset_index()
    if 'CÓD DA FAMILIA' in dados_reset.columns:
        familias_base = dados_reset['CÓD DA FAMILIA']
    else:
        raise KeyError("Coluna 'CÓD DA FAMILIA' não encontrada em dados (nem no índice).")

familias_existentes = familias_base.apply(normaliza_familia)
familias_existentes = familias_existentes[familias_existentes.notna()]
set_familias_existentes = set(familias_existentes.tolist())

novos['Cod da Familia Norm'] = novos['Cod da Familia'].apply(normaliza_familia)

novos_familia_existente = novos[novos['Cod da Familia Norm'].isin(set_familias_existentes)].copy()
novos_familia_nova = novos[~novos['Cod da Familia Norm'].isin(set_familias_existentes)].copy()

colunas_saida = ['Cod da Familia', 'Codigo', 'Dependencia', 'Nome', 'CPF', 'Data Nasc.']

def montar_planilha(df):
    if df.empty:
        return pd.DataFrame(columns=colunas_saida)

    saida = pd.DataFrame({
        'Cod da Familia': df['Cod da Familia'],
        'Codigo': df['Codigo'],
        'Dependencia': pd.NA,
        'Nome': df['Nome_norm'],
        'CPF': pd.NA,
        'Data Nasc.': df['Data Nasc.']
    })

    saida = saida.sort_values(by=['Cod da Familia', 'Nome'], kind='stable')

    linhas = []
    prev_familia = None
    for _, row in saida.iterrows():
        familia_atual = row['Cod da Familia']
        if prev_familia is not None and familia_atual != prev_familia:
            linhas.append({col: pd.NA for col in colunas_saida})
        linhas.append(row.to_dict())
        prev_familia = familia_atual

    return pd.DataFrame(linhas, columns=colunas_saida)

planilha_familia_nova = montar_planilha(novos_familia_nova)
planilha_familia_existente = montar_planilha(novos_familia_existente)

planilha_familia_nova.to_excel('../novos_beneficiarios_familia_nova.xlsx', index=False)
planilha_familia_existente.to_excel('../novos_beneficiarios_familia_existente.xlsx', index=False)
print("Planilhas geradas:")
print("- ../novos_beneficiarios_familia_nova.xlsx")
print("- ../novos_beneficiarios_familia_existente.xlsx")

### 2.4 Verificação de Nomes Sobrando (Dados -> Mensalidades)
Verifica nomes que estão na nossa planilha de dados mas não aparecem na planilha de mensalidades (quem não está sendo cobrado).

In [ ]:
nomes_mensalidades = (
    mensalidades['Nome']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)
nomes_dados = (
    dados['NOME']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

nomes_mensalidades = nomes_mensalidades[nomes_mensalidades.notna() & (nomes_mensalidades != '')]
nomes_dados = nomes_dados[nomes_dados.notna() & (nomes_dados != '')]

set_mensalidades = set(nomes_mensalidades.tolist())
nomes_nao_encontrados_dados = [nome for nome in nomes_dados.tolist() if nome not in set_mensalidades]

if nomes_nao_encontrados_dados:
    print("Names in 'dados' not found in 'mensalidades':")
    for nome in nomes_nao_encontrados_dados:
        print(nome)
else:
    print("All names in 'dados' are present in 'mensalidades'.")

### 2.5 Verificação de Duplicatas
Verifica se há nomes duplicados nas planilhas.

In [ ]:
duplicate_names = dados[dados['NOME'].notna() & dados['NOME'].duplicated(keep=False)]

if not duplicate_names.empty:
    print("Nomes duplicados encontrados na planilha 'dados' (excluindo NaN):")
    display(duplicate_names)
else:
    print("Não há nomes duplicados na planilha 'dados' (excluindo NaN).")

In [ ]:
dados_mensalidade_not_null = dados[dados['MENSALIDADE'].notna()].copy()

nomes_dados = (
    dados_mensalidade_not_null['NOME']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

nomes_mensalidades = (
    mensalidades['Nome']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

nomes_dados = nomes_dados[nomes_dados.notna() & (nomes_dados != '')]
nomes_mensalidades = nomes_mensalidades[nomes_mensalidades.notna() & (nomes_mensalidades != '')]

set_mensalidades = set(nomes_mensalidades.tolist())

names_in_dados_not_in_mensalidades = [
    nome for nome in nomes_dados.tolist() if nome not in set_mensalidades
]

if names_in_dados_not_in_mensalidades:
    print("Nomes presentes em 'dados' (com MENSALIDADE não nula) e não presentes em 'mensalidades':")
    for nome in names_in_dados_not_in_mensalidades:
        print(nome)
else:
    print("Não há nomes em 'dados' (com MENSALIDADE não nula) que não estejam em 'mensalidades'.")

In [ ]:
mensalidades_filtered = mensalidades.dropna(subset=['Data Nasc.'])

nomes_norm = (
    mensalidades_filtered['Nome']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

name_counts = nomes_norm.value_counts()

duplicate_names_counts = name_counts[name_counts > 1]

if not duplicate_names_counts.empty:
    print("Contagem de nomes duplicados encontrados na planilha 'mensalidades' (excluindo Data Nasc. nulo):")
    display(duplicate_names_counts)
else:
    print("Não há nomes duplicados na planilha 'mensalidades' (excluindo Data Nasc. nulo).")

In [ ]:
mensalidades_filtered = mensalidades.dropna(subset=['Data Nasc.'])

nomes_norm = (
    mensalidades_filtered['Nome']
    .astype('string')
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

duplicate_names_mensalidades = mensalidades_filtered[nomes_norm.duplicated(keep=False)]

if not duplicate_names_mensalidades.empty:
    print("Nomes duplicados encontrados na planilha 'mensalidades' (excluindo Data Nasc. nulo):")
    display(duplicate_names_mensalidades)
else:
    print("Não há nomes duplicados na planilha 'mensalidades' (excluindo Data Nasc. nulo).")